In [64]:
import pandas as pd
import re
import warnings 

warnings.filterwarnings('ignore') 

In [65]:
raw_df = pd.read_csv('gun-violence-data.csv')
nacheck_df = raw_df[['date', 'state', 'n_killed', 'n_injured', 'gun_type', 'n_guns_involved', 
             'participant_age', 'participant_gender', 'participant_type']]

nacheck_df.isnull().sum()

date                      0
state                     0
n_killed                  0
n_injured                 0
gun_type              99451
n_guns_involved       99451
participant_age       92298
participant_gender    36362
participant_type      24863
dtype: int64

In [66]:
raw_df = pd.read_csv('gun-violence-data.csv')
df = raw_df[['date', 'state', 'n_killed', 'n_injured', 'gun_type', 'n_guns_involved']]
participants = raw_df[['participant_age', 'participant_gender', 'participant_type']]
participants

,participant_age,participant_gender,participant_type
0,0::20,0::Male||1::Male||3::Male||4::Female,0::Victim||1::Victim||2::Victim||3::Victim||4:...
1,0::20,0::Male,0::Victim||1::Victim||2::Victim||3::Victim||4:...
2,0::25||1::31||2::33||3::34||4::33,0::Male||1::Male||2::Male||3::Male||4::Male,0::Subject-Suspect||1::Subject-Suspect||2::Vic...
3,0::29||1::33||2::56||3::33,0::Female||1::Male||2::Male||3::Male,0::Victim||1::Victim||2::Victim||3::Subject-Su...
4,0::18||1::46||2::14||3::47,0::Female||1::Male||2::Male||3::Female,0::Victim||1::Victim||2::Victim||3::Subject-Su...
...,...,...,...
239672,0::25,0::Female,0::Subject-Suspect
239673,1::21,0::Male||1::Male,0::Victim||1::Subject-Suspect
239674,0::21,0::Male,0::Victim
239675,0::42,0::Male,0::Victim


# Cleaning Participants

Creating seperate columns for the victim(s) and the shooter(s)

In [67]:
def parse_column(data_str):
    if pd.isna(data_str):
        return {}
    return {int(part.split("::")[0] if "::" in part else part.split(":")[0]): 
            part.split("::")[1] if "::" in part else part.split(":")[1] 
            for part in data_str.split("||")}

def process_dataframe(df):
    df['suspect_age'] = ''
    df['victim_age'] = ''
    df['suspect_gender'] = ''
    df['victim_gender'] = ''
    
    for index, row in df.iterrows():
        age_dict = parse_column(row['participant_age'])
        gender_dict = parse_column(row['participant_gender'])
        types_list = parse_column(row['participant_type'])

        suspect_age_list = []
        victim_age_list = []
        suspect_gender_list = []
        victim_gender_list = []

        for idx, role in types_list.items():
            if role == "Subject-Suspect":
                suspect_age_list.append(age_dict.get(idx, 'Unknown'))
                suspect_gender_list.append(gender_dict.get(idx, 'Unknown'))
            else:
                victim_age_list.append(age_dict.get(idx, 'Unknown'))
                victim_gender_list.append(gender_dict.get(idx, 'Unknown'))
        
        df.at[index, 'suspect_age'] = ', '.join(suspect_age_list)
        df.at[index, 'victim_age'] = ', '.join(victim_age_list)
        df.at[index, 'suspect_gender'] = ', '.join(suspect_gender_list)
        df.at[index, 'victim_gender'] = ', '.join(victim_gender_list)
    
    return df

processed_df = process_dataframe(participants)
cleaned_participants = processed_df[['suspect_age', 'victim_age', 'suspect_gender', 'victim_gender']]
cleaned_participants

,suspect_age,victim_age,suspect_gender,victim_gender
0,Unknown,"20, Unknown, Unknown, Unknown",Female,"Male, Male, Unknown, Male"
1,Unknown,"20, Unknown, Unknown, Unknown",Unknown,"Male, Unknown, Unknown, Unknown"
2,"25, 31","33, 34, 33","Male, Male","Male, Male, Male"
3,33,"29, 33, 56",Male,"Female, Male, Male"
4,47,"18, 46, 14",Female,"Female, Male, Male"
...,...,...,...,...
239672,25,,Female,
239673,21,Unknown,Male,Male
239674,,21,,Male
239675,,42,,Male


# Cleaning The Rest

Combining participants and cleaning the rest of the data

In [68]:
cleaned_df = pd.concat([df, cleaned_participants], axis=1)
cleaned_df.isnull().sum()

date                   0
state                  0
n_killed               0
n_injured              0
gun_type           99451
n_guns_involved    99451
suspect_age            0
victim_age             0
suspect_gender         0
victim_gender          0
dtype: int64

In [69]:
cleaned_df['gun_type'] = cleaned_df['gun_type'].fillna('Unknown')
cleaned_df['n_guns_involved'] = cleaned_df['n_guns_involved'].fillna('Unknown')
cleaned_df.isnull().sum()

date               0
state              0
n_killed           0
n_injured          0
gun_type           0
n_guns_involved    0
suspect_age        0
victim_age         0
suspect_gender     0
victim_gender      0
dtype: int64

In [70]:
def clean_gun_type(gun_type):
    if pd.isna(gun_type):
        return 'Unknown'
    
    gun_type = str(gun_type)
    
    if gun_type == 'Unknown':
        return 'Unknown'

    parts = gun_type.split('||')
    cleaned_types = []

    for part in parts:
        if '::' in part:
            index, type_desc = part.split('::')
        else:
            type_desc = 'Unknown'

        bracketed_type = re.findall(r'\[(.*?)\]', type_desc)
        if bracketed_type:
            cleaned_types.append(bracketed_type[0])
        else:
            cleaned_types.append(type_desc)

    return ', '.join(cleaned_types)

cleaned_df['gun_type'] = df['gun_type'].apply(clean_gun_type)
cleaned_df

,date,state,n_killed,n_injured,gun_type,n_guns_involved,suspect_age,victim_age,suspect_gender,victim_gender
0,2013-01-01,Pennsylvania,0,4,Unknown,Unknown,Unknown,"20, Unknown, Unknown, Unknown",Female,"Male, Male, Unknown, Male"
1,2013-01-01,California,1,3,Unknown,Unknown,Unknown,"20, Unknown, Unknown, Unknown",Unknown,"Male, Unknown, Unknown, Unknown"
2,2013-01-01,Ohio,1,3,"Unknown, Unknown",2.0,"25, 31","33, 34, 33","Male, Male","Male, Male, Male"
3,2013-01-05,Colorado,4,0,Unknown,Unknown,33,"29, 33, 56",Male,"Female, Male, Male"
4,2013-01-07,North Carolina,2,2,"Handgun, Handgun",2.0,47,"18, 46, 14",Female,"Female, Male, Male"
...,...,...,...,...,...,...,...,...,...,...
239672,2018-03-31,Louisiana,0,0,Unknown,1.0,25,,Female,
239673,2018-03-31,Louisiana,1,0,Unknown,1.0,21,Unknown,Male,Male
239674,2018-03-31,Louisiana,0,1,Unknown,1.0,,21,,Male
239675,2018-03-31,Texas,1,0,Unknown,1.0,,42,,Male


# Adding Gun Law Strictness by State

Law strictness labels grabbed from: https://www.datapandas.org/ranking/strictest-gun-laws-by-state

In [76]:
gun_law_data = {
    'Region': ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 
               'Connecticut', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 
               'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 
               'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 
               'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 
               'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 
               'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 
               'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 
               'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 
               'Wisconsin', 'Wyoming'],
    'Gun Law Strength': ['F', 'F', 'F', 'F', 'A', 'B', 'A-', 'B', 'C-', 'F', 'A-', 'F', 
                         'A-', 'F', 'F', 'F', 'F', 'F', 'F', 'A-', 'A-', 'C+', 'C+', 'F', 
                         'F', 'F', 'C', 'C+', 'F', 'A', 'C+', 'A-', 'C', 'F', 'F', 'F', 
                         'B+', 'B-', 'B+', 'F', 'F', 'F', 'F', 'F', 'C-', 'B', 'B+', 
                         'F', 'D+', 'F']
}


gun_law_df = pd.DataFrame(gun_law_data)
gun_law_map = dict(zip(gun_law_df['Region'], gun_law_df['Gun Law Strength']))
cleaned_df['Gun Law Strictness'] = cleaned_df['state'].map(gun_law_map)
cleaned_df

,date,state,n_killed,n_injured,gun_type,n_guns_involved,suspect_age,victim_age,suspect_gender,victim_gender,Gun Law Strictness
0,2013-01-01,Pennsylvania,0,4,Unknown,Unknown,Unknown,"20, Unknown, Unknown, Unknown",Female,"Male, Male, Unknown, Male",B-
1,2013-01-01,California,1,3,Unknown,Unknown,Unknown,"20, Unknown, Unknown, Unknown",Unknown,"Male, Unknown, Unknown, Unknown",A
2,2013-01-01,Ohio,1,3,"Unknown, Unknown",2.0,"25, 31","33, 34, 33","Male, Male","Male, Male, Male",F
3,2013-01-05,Colorado,4,0,Unknown,Unknown,33,"29, 33, 56",Male,"Female, Male, Male",B
4,2013-01-07,North Carolina,2,2,"Handgun, Handgun",2.0,47,"18, 46, 14",Female,"Female, Male, Male",C
...,...,...,...,...,...,...,...,...,...,...,...
239672,2018-03-31,Louisiana,0,0,Unknown,1.0,25,,Female,,F
239673,2018-03-31,Louisiana,1,0,Unknown,1.0,21,Unknown,Male,Male,F
239674,2018-03-31,Louisiana,0,1,Unknown,1.0,,21,,Male,F
239675,2018-03-31,Texas,1,0,Unknown,1.0,,42,,Male,F


In [78]:
cleaned_df.to_csv('cleaned-gun-violence-data.csv')